### Fleiss' Kappa

In [ ]:
from metrics import evaluate_consistency
import os

root_path = ""
model_name = ""
for task in ["math100", "aime2024", "IMO"]:
    a = evaluate_consistency(os.path.join(root_path, task, model_name))
    print(f"Fleiss' Kappa - {task}: ", a)